In [1]:
%matplotlib inline

In [ ]:
# Deep Q Learning / Frozen Lake / Not Slippery / 4x4
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Sequential
from collections import deque

custom_map = [
    'FFFF',
    'FHFH',
    'FFFH',
    'HFFG'
]

env = gym.make("FrozenLake-v0", desc=custom_map, is_slippery=False)
train_episodes=4000
test_episodes=100
max_steps=300
state_size = env.observation_space.n
action_size = env.action_space.n
batch_size=32

class Agent:
    def __init__(self, state_size, action_size):
        self.memory = deque(maxlen=2500)
        self.learning_rate=0.001
        self.epsilon=1
        self.max_eps=1
        self.min_eps=0.01
        self.eps_decay = 0.001/3
        self.gamma=0.9
        self.state_size= state_size
        self.action_size= action_size
        self.epsilon_lst=[]
        self.model = self.buildmodel()

    def buildmodel(self):
        model=Sequential()
        model.add(Dense(10, input_dim=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, new_state, reward, done, state, action):
        self.memory.append((new_state, reward, done, state, action))

    def action(self, state):
        if np.random.rand() > self.epsilon:
            return np.random.randint(0,4)
        return np.argmax(self.model.predict(state))

    def pred(self, state):
        return np.argmax(self.model.predict(state))

    def replay(self,batch_size):
        minibatch=random.sample(self.memory, batch_size)
        for new_state, reward, done, state, action in minibatch:
            target= reward
            if not done:
                target=reward + self.gamma* np.amax(self.model.predict(new_state))
            target_f= self.model.predict(state)
            target_f[0][action]= target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.min_eps:
            self.epsilon=(self.max_eps - self.min_eps) * np.exp(-self.eps_decay*episode) + self.min_eps

        self.epsilon_lst.append(self.epsilon)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

agent=Agent(state_size, action_size)

reward_lst=[]
for episode in range(train_episodes):
    state= env.reset()
    state_arr=np.zeros(state_size)
    state_arr[state] = 1
    state= np.reshape(state_arr, [1, state_size])
    reward = 0
    done = False
    for t in range(max_steps):
        # env.render()
        action = agent.action(state)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[new_state] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])
        agent.add_memory(new_state, reward, done, state, action)
        state= new_state

        if done:
            print(f'Episode: {episode:4}/{train_episodes} and step: {t:4}. Eps: {float(agent.epsilon):.2}, reward {reward}')
            break

    reward_lst.append(reward)

    if len(agent.memory)> batch_size:
        agent.replay(batch_size)

print(' Train mean % score= ', round(100*np.mean(reward_lst),1))

# test
test_wins=[]
for episode in range(test_episodes):
    state = env.reset()
    state_arr=np.zeros(state_size)
    state_arr[state] = 1
    state= np.reshape(state_arr, [1, state_size])
    done = False
    reward=0
    state_lst = []
    state_lst.append(state)
    print('******* EPISODE ',episode, ' *******')

    for step in range(max_steps):
        action = agent.pred(state)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[new_state] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])
        state = new_state
        state_lst.append(state)
        if done:
            print(reward)
            # env.render()
            break

    test_wins.append(reward)
env.close()

print(' Test mean % score= ', int(100*np.mean(test_wins)))

fig=plt.figure(figsize=(10,12))
matplotlib.rcParams.clear()
matplotlib.rcParams.update({'font.size': 22})
plt.subplot(311)
plt.scatter(list(range(len(reward_lst))), reward_lst, s=0.2)
plt.title('4x4 Frozen Lake Result(DQN) \n \nTrain Score')
plt.ylabel('Score')
plt.xlabel('Episode')

plt.subplot(312)
plt.scatter(list(range(len(agent.epsilon_lst))), agent.epsilon_lst, s=0.2)
plt.title('Epsilon')
plt.ylabel('Epsilon')
plt.xlabel('Episode')

plt.subplot(313)
plt.scatter(list(range(len(test_wins))), test_wins, s=0.5)
plt.title('Test Score')
plt.ylabel('Score')
plt.xlabel('Episode')
plt.ylim((0,1.1))
plt.savefig('result.png',dpi=300)
plt.show()

Episode:    0/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    1/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    2/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    3/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    4/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    5/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    6/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    7/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    8/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:    9/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   10/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   11/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   12/4000 and step:   19. Eps: 1.0, reward 0.0
Episode:   13/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   14/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   15/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   16/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   17/4000 and step:   99. Eps: 0.99, re